In [1]:
# Mount Google Drive
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download Dataset
!wget https://ndownloader.figshare.com/articles/1512427/versions/5

In [ ]:
# Unzip Dataset
!unzip 5
!rm 5
!cat brainTumorDataPublic_* > brainTumorDataPublic_temp.zip
!zip -FF brainTumorDataPublic_temp.zip --out data.zip
!rm brainTumorDataPublic_*
!unzip data.zip -d data
!rm data.zip

In [ ]:
# Preprocess and Convert Dataset to Numpy Format 
!cp -r "./data" "/content/drive/My Drive/Cog Neuro Project/Datasets/Task 2"
!pip install hdf5storage
!python ./matlab_to_numpy.py "/content/drive/My Drive/Cog Neuro Project/Datasets/Task 2" --image-dimension 224

In [3]:
# Set Path
path = "/content/drive/My Drive/Cog Neuro Project/Datasets/Task 2/"

In [2]:
# Necessary Imports
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
# Load Labels, Images and Tumor Masks
# Tumor Mask - Binary Data where Tumor Presence is True and Absence is False
Y = np.load(path + "labels.npy")
X = np.load(path + "images.npy")
X_mask = np.load(path + "masks.npy")
# Zero Index the Labels
Y = Y - 1

In [5]:
# Create Train and Test Datasets
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,shuffle=True,random_state=42,test_size=0.1)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(2757, 224, 224) (307, 224, 224) (2757,) (307,)


In [6]:
# Perform One Hot Encoding
X_train = X_train.reshape(2757,224,224,1)
X_test = X_test.reshape(307,224,224,1)
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [7]:
# Construct CNN Model
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(224,224,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43264)             0

In [ ]:
# Train CNN Model with Validation Based Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
model.fit(X_train, Y_train,validation_data=(X_test,Y_test), epochs=100,callbacks=[es])

In [ ]:
# Print Results
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# Save and Load Model
model.save('/content/model')
temp = keras.models.load_model('/content/model')
print(temp.summary())

In [11]:
# Test Loaded Model
#temp = keras.models.load_model('/content/content/model')
scores = temp.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (temp.metrics_names[1], scores[1]*100))

accuracy: 91.86%


In [ ]:
# Zip Model with Weights
!zip -r /content/model.zip /content/model

In [ ]:
# Download Model with Weights
files.download("/content/model.zip")